In [1]:
import streamlit as st
import pandas as pd
import plotly.express as px
import altair as alt
import numpy as np
#from st_aggrid import GridOptionsBuilder, AgGrid, GridUpdateMode, DataReturnMode

import openpyxl
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
from itertools import islice


In [2]:
show_fig = False
input_filename_scenario_01 = 'tenancy_list_scenario_2023_v01C.xlsx'
input_filename_scenario_02 = 'tenancy_list_scenario_2023_v01C.xlsx'

#input_filename_scenario_02 = 'tenancy_list_scenario_01C.xlsx'
output_filename = 'Revenue 2024 2033 simulation yearly.xlsx'

# Scenario 1

In [3]:

# open excel file
excel_file_01 = input_filename_scenario_01
wb_01 = openpyxl.load_workbook(excel_file_01, data_only=True)

# getting all sheets
sheets = wb_01.sheetnames

# getting a particular sheet
sheet_params_01 = wb_01["params"]
sheet_data_01 = wb_01["tenant"]

# reading parameters
date_start_01 = sheet_params_01["C4"].value
date_end_01 = sheet_params_01['C5'].value
area_rentable_office_01 = sheet_params_01["C6"].value

c:\Users\samad\miniconda3\envs\office-revenues\Lib\site-packages\openpyxl\reader\excel.py:237: UserWarning: Unknown extension is not supported and will be removed
  ws_parser.bind_all()
c:\Users\samad\miniconda3\envs\office-revenues\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


# Scenario 2

In [4]:

# open excel file
excel_file_02 = input_filename_scenario_02
wb_02 = openpyxl.load_workbook(excel_file_02, data_only=True)

# getting all sheets
sheets = wb_02.sheetnames

# getting a particular sheet
sheet_params_02 = wb_02["params"]
sheet_data_02 = wb_02["tenant"]

# reading parameters
date_start_02 = sheet_params_02["C4"].value
date_end_02 = sheet_params_02['C5'].value
area_rentable_office_02 = sheet_params_02["C6"].value

# Definition of read excel worksheet

In [5]:

# Function: Read a sheet into dataframe
# sheet_to_read is type openpyxl.workbook.Workbook
def read_worksheet_into_dataframe(sheet_to_read):
    data = sheet_to_read.values
    cols = next(data)[1:]
    data = list(data)
    #idx = [r[0] for r in data]
    data = (islice(r, 1, None) for r in data)
    df_data = pd.DataFrame(data, columns=cols)

    # drop rows where column Tenant is Null/NA
    df_data = df_data[df_data['Tenant'].notna()]
    return df_data


# Definition of Calculate Revenue

In [6]:

# Function: Calculate revenue
def calculate_revenue(date_start, date_end, area_rentable_office, df_data, product_type, report_sum):

    # generate reporting daterange based on start and end dates
    my_reporting_date_range = pd.date_range(start=date_start, end=date_end, freq='MS')

    # Create report dataframe using datetime index for period
    df_report_rental_charge = pd.DataFrame(my_reporting_date_range, columns=['date'])   # Rental revenue
    df_report_sc_charge = pd.DataFrame(my_reporting_date_range, columns=['date'])       # Service charge revenue
    df_report_sc_charge_rate = pd.DataFrame(my_reporting_date_range, columns=['date'])       # Service charge rate
    df_report_occupancy = pd.DataFrame(my_reporting_date_range, columns=['date'])       # Occupancy rate

    # Set 'date' as the index
    df_report_rental_charge = df_report_rental_charge.set_index('date')
    df_report_sc_charge = df_report_sc_charge.set_index('date')
    df_report_occupancy = df_report_occupancy.set_index('date')
    df_pid_tenant_name_mapping = pd.DataFrame()


    # Calculate how much is SC rate based on months and years
    # April 2022: 75000
    # April 2024: 80000
    # April 2026: 85000

    sc_data = [
        [datetime(2018, 4, 1), 65000.00],
        [datetime(2020, 4, 1), 70000.00],
        [datetime(2022, 4, 1), 70000.00],
        [datetime(2025, 4, 1), 75000.00],
        [datetime(2028, 4, 1), 80000.00],
        [datetime(2031, 4, 1), 85000.00],
        [datetime(2034, 4, 1), 90000.00],
    ]

    # select rows according to product_type passed in as parameter into this function
    options = [product_type]
    df_by_product = df_data[df_data['Product_Type'].isin(options)]

    substring = '-'

    # start revenue calculation
    # iterate each row of dataframe
    for index, row in df_by_product.iterrows():
        vacant = False
    
        data_area = row['Area']
        data_rental_rate = row['Rental_Rate']
        data_sc_rate = row['SC_Rate']
        data_tenant_name = row['Tenant']
        charge_type = row['Chg_Type']

        if (data_area == None) or (substring in str(data_area)):
            data_area = 0.0

        if (data_rental_rate == None) or (substring in str(data_rental_rate)):
            data_rental_rate = 0.0

        if (data_sc_rate == None) or (substring in str(data_sc_rate)):
            data_sc_rate = 0.0

        # calculate monthly rental charge
        if charge_type == 'A': # if calculation based on area
            calc_rental_charge = data_area * data_rental_rate
        else: # else lumpsum
            calc_rental_charge = data_rental_rate
        
        # calculate monthly service charge
        calc_service_charge = data_area * data_sc_rate

        start = row["Start"]
        end = row["End"]

        # if end is empty, it means it is vacant
        if (pd.isna(end)) or (end == '-'):
            vacant = True
            end = date_end

        # generate name for the column
        str_level = str(row['Floor']).split('.')[0]
        str_zone = str(row['Zone'])
        if (str_level == 'None'):
            str_level = 'NA'
        if (str_zone == 'None'):
            str_zone = 'NA'
        str_sep = '-'
        str_temps = [str(str_level), str(str_zone), str(index)]
        str_column_name = str_sep.join(str_temps)
        dict01 = {'pid':[str_column_name], 'cust_name':[str_level + '-' + str_zone + '   ' + data_tenant_name]}
        df_temp = pd.DataFrame(dict01)
        df_pid_tenant_name_mapping = pd.concat([df_pid_tenant_name_mapping, df_temp], ignore_index=True)

        # if start is empty, set start equals to date_start parameter
        if (pd.isna(start)) or (start == '-'):
            start = date_start
            
        tenant_date_range = pd.date_range(start=start, end=end, freq='MS')

        if not vacant:
            # create daterange series for tenant period
            tenant_date_range = pd.date_range(start=start, end=end, freq='MS')

            # generate rental charge dataframe
            df_tenant_rental_charge = pd.DataFrame(tenant_date_range, columns=['date']) # create new df with only 1 column called 'date' and fill with the date range from lcd to led
            df_tenant_rental_charge = df_tenant_rental_charge.set_index('date') # set 'date' as the index
            df_tenant_rental_charge[str_column_name] = calc_rental_charge # add a new column called <num> and fill all with the rental_charge value

            # join with report dataframe based on date
            df_report_rental_charge = df_report_rental_charge.join(df_tenant_rental_charge, how="left")

            # ORIGINAL ----- START
            # # generate service charge report
            # df_sc_data = pd.DataFrame(data=sc_data, columns=['date', 'sc']).set_index('date')
            # sc_date_range = pd.date_range(start=df_sc_data.index.values.min(), end=df_sc_data.index.values.max(), freq='MS')
            # df_sc = pd.DataFrame(sc_date_range, columns=['date'])
            # df_sc = df_sc.set_index('date')

            # for mydate in df_sc_data.index:
            #     date1 = mydate
            #     date2 = mydate+relativedelta(years=2)-relativedelta(days=1)
            # #print('{0} - {1}'.format(date1.strftime('%Y-%m-%d'), date2.strftime('%Y-%m-%d')))
            #     if data_sc_rate == 0:
            #         df_sc.loc[date1.strftime('%Y-%m-%d'):date2.strftime('%Y-%m-%d'), str_column_name] = 0
            #     elif data_sc_rate == 84100:
            #         df_sc.loc[date1.strftime('%Y-%m-%d'):date2.strftime('%Y-%m-%d'), str_column_name] = 84100
            #     else:
            #         df_sc.loc[date1.strftime('%Y-%m-%d'):date2.strftime('%Y-%m-%d'), str_column_name] = df_sc_data.loc[mydate, 'sc']
            # ORIGINAL ----- END

            # NEW ----- START
            # generate service charge report
            # df_sc_data = pd.DataFrame(data=sc_data, columns=['date', 'sc']).set_index('date')
            df_sc_data = pd.DataFrame(data=sc_data, columns=['date', 'sc'])
            df_sc_data.reset_index(inplace=True)
            sc_date_range = pd.date_range(start=df_sc_data.date.values.min(), end=df_sc_data.date.values.max(), freq='MS')
            df_sc = pd.DataFrame(sc_date_range, columns=['date'])
            df_sc = df_sc.set_index('date')

            # data_sc_rate = 70000
            # str_column_name = 'TEST'

            length = len(df_sc_data.index)
            for i in range(length):
                if i < length-1:
                    date1 = df_sc_data.iloc[i,1]
                    date2 = df_sc_data.iloc[i+1,1]
                    sc_rate = df_sc_data.iloc[i,2]
                    # print(f"{date1} to {date2} is {sc_rate}" )
                    if data_sc_rate == 0:
                        df_sc.loc[date1.strftime('%Y-%m-%d'):date2.strftime('%Y-%m-%d'), str_column_name] = 0
                    elif data_sc_rate == 84100:
                        df_sc.loc[date1.strftime('%Y-%m-%d'):date2.strftime('%Y-%m-%d'), str_column_name] = 84100
                    else:
                        df_sc.loc[date1.strftime('%Y-%m-%d'):date2.strftime('%Y-%m-%d'), str_column_name] = df_sc_data.loc[i, 'sc']
            # ORIGINAL ----- END

            df_sc_rate_temp = df_sc
            df_sc = df_sc * data_area

            df_tenant_service_charge = pd.DataFrame(tenant_date_range, columns=['date'])
            df_tenant_service_charge = df_tenant_service_charge.set_index('date')
            df_tenant_service_charge = df_tenant_service_charge.join(df_sc, how='left')
            df_report_sc_charge = df_report_sc_charge.join(df_tenant_service_charge, how="left")

            df_tenant_service_charge_rate = pd.DataFrame(tenant_date_range, columns=['date'])
            df_tenant_service_charge_rate = df_tenant_service_charge_rate.set_index('date')
            df_tenant_service_charge_rate = df_tenant_service_charge_rate.join(df_sc_rate_temp, how='left')
            df_report_sc_charge_rate = df_report_sc_charge_rate.join(df_tenant_service_charge_rate, how="left")
            # generate occupancy report
            df_tenant_occupancy = pd.DataFrame(tenant_date_range, columns=['date'])
            df_tenant_occupancy = df_tenant_occupancy.set_index('date')
            df_tenant_occupancy[str_column_name] = data_area
            df_report_occupancy = df_report_occupancy.join(df_tenant_occupancy, how="left")

        
        else: # if vacant
            # generate occupancy report
            df_tenant_occupancy = pd.DataFrame(tenant_date_range, columns=['date'])
            df_tenant_occupancy = df_tenant_occupancy.set_index('date')
            df_tenant_occupancy[str_column_name] = -1.0
            df_report_occupancy = df_report_occupancy.join(df_tenant_occupancy, how="left")


    # Clean dataframes
    df_report_rental_charge.fillna(0, inplace=True)
    df_report_sc_charge.fillna(0, inplace=True)
    df_report_occupancy.fillna(0, inplace=True)

    # sum each rows
    df_report_rental_charge['sum'] = df_report_rental_charge.sum(axis=1)
    df_report_sc_charge['sum'] = df_report_sc_charge.sum(axis=1)
    df_report_occupancy['sum'] = df_report_occupancy.sum(axis=1)
    
    # create summary report
    df_sum = pd.DataFrame()
    df_sum['Rental'] = df_report_rental_charge.resample(report_sum).sum()['sum']
    df_sum['SC'] = df_report_sc_charge.resample(report_sum).sum()['sum']
    df_sum['Total'] = df_sum.sum(axis=1)

    df_sum['Occ'] = df_report_occupancy.resample(report_sum).mean()['sum']
    df_sum['OccPct'] = df_sum['Occ']/area_rentable_office
    df_sum.reset_index(inplace=True)
    return df_sum, df_report_rental_charge, df_report_sc_charge, df_report_occupancy, df_report_sc_charge_rate, df_pid_tenant_name_mapping


In [7]:
df_data_01 = read_worksheet_into_dataframe(sheet_data_01)
df_data_02 = read_worksheet_into_dataframe(sheet_data_02)

## Calculate Monthly and put it in TE Office (m) sheet

In [8]:
df_sum_01, df_report_rental_charge_01, df_report_sc_charge_01, df_report_occupancy_01, df_report_sc_charge_rate_01, df_pid_tenant_name_mapping_01 = calculate_revenue(date_start_01, date_end_01, area_rentable_office_01, df_data_01, 'Office', 'M')
df_sum_02, df_report_rental_charge_02, df_report_sc_charge_02, df_report_occupancy_02, df_report_sc_charge_rate_02, df_pid_tenant_name_mapping_02 = calculate_revenue(date_start_02, date_end_02, area_rentable_office_02, df_data_02, 'Office', 'M')

### This is how to remove NaT

In [9]:
df_data_01.head(5).replace({np.nan: None},)

,Location,Floor,Zone,Product_Type,Group,Tenant,Area,Chg_Type,Start,End,...,Notes_1,None,None,None,None,None,None,None,None,None
0,TE,PH,All,Office,Y,Medco Energi International,1183.46,A,2016-01-01,2034-06-30,...,None,None,None,None,None,None,None,None,None,None
1,TE,55,All,Office,Y,Medco Energi International,1556.52,A,2016-01-01,2029-06-30,...,None,None,None,None,None,None,None,None,None,None
2,TE,55,All,Office,Y,Medco Energi International,1556.52,A,2029-07-01,2034-06-30,...,None,None,None,None,None,None,None,1556.52,607042800.0,None
3,TE,53,All,Office,Y,Medco Energi International,1625.82,A,2016-01-01,2029-06-30,...,None,None,None,None,None,None,None,1625.82,589359750.0,None
4,TE,53,All,Office,Y,Medco Energi International,1625.82,A,2029-07-01,2034-06-30,...,None,None,None,None,None,None,None,None,None,None


### Format Style for dataframe

In [10]:
def format_time_nat(t, fmt='{:%Y.%m}'):
    try:
        return fmt.format(t)
    except ValueError:
        return t

df_data_01.head(5).style.format(
    {
        "Area": "{:20,.2f}",
        "Rental_Rate": "{:20,.2f}",
        "SC_Rate": "{:20,.0f}",
        "Start": format_time_nat,
        "End": format_time_nat,
        },
    # {}
)

,Location,Floor,Zone,Product_Type,Group,Tenant,Area,Chg_Type,Start,End,Rental_Rate,SC_Rate,Notes_1,None,None,None,None,None,None,None,None,None
0,TE,PH,All,Office,Y,Medco Energi International,"1,183.46",A,2016.01,2034.06,0.00,"84,100",None,None,None,None,nan,None,None,nan,nan,nan
1,TE,55,All,Office,Y,Medco Energi International,"1,556.52",A,2016.01,2029.06,"362,500.00","84,100",None,None,None,None,nan,None,None,nan,nan,nan
2,TE,55,All,Office,Y,Medco Energi International,"1,556.52",A,2029.07,2034.06,"390,000.00","84,100",None,None,None,None,nan,None,None,1556.520000,607042800.000000,nan
3,TE,53,All,Office,Y,Medco Energi International,"1,625.82",A,2016.01,2029.06,"362,500.00","84,100",None,None,None,None,nan,None,None,1625.820000,589359750.000000,nan
4,TE,53,All,Office,Y,Medco Energi International,"1,625.82",A,2029.07,2034.06,"390,000.00","84,100",None,None,None,None,nan,None,None,nan,nan,nan


### Write to Excel file for presentation

In [11]:
# write to excel file
import xlwings as xw
wb = xw.Book(output_filename)
my_sheet = wb.sheets['TE Office (m)']
my_sheet.clear_contents()


#### Uncomment below to see monthly occupancy

In [12]:
# import xlwings as xw
# wb10 = xw.Book()
# # sht1 = wb.sheets['tenant']
# sheet10 = wb10.sheets[0]
# sheet10.range('A1').value = df_report_occupancy_01

### Scenario 1

In [13]:
df_to_show = df_sum_01.copy()

my_sheet.range('B3').value = df_to_show.transpose()
my_sheet.range('C4:M4').number_format = 'mm-yyyy'
my_sheet.range('3:4').font.bold = True
my_sheet.range('B:B').font.bold = True
my_sheet.range('7:7').font.bold = True
my_sheet.range('C5:M8').number_format = '0,00#'
my_sheet.range('C9:M9').number_format = '0.00%'


### Scenario 2

In [14]:
# df_to_show = df_sum_02.copy()

# my_sheet.range('B12').value = df_to_show.transpose()
# my_sheet.range('C13:M13').number_format = 'yyyy'
# my_sheet.range('12:13').font.bold = True
# my_sheet.range('B:B').font.bold = True
# my_sheet.range('16:16').font.bold = True
# my_sheet.range('C14:M17').number_format = '0,00#'
# my_sheet.range('C18:M18').number_format = '0.00%'


## Calculate Yearly

In [15]:
df_sum_01, df_report_rental_charge_01, df_report_sc_charge_01, df_report_occupancy_01, df_report_sc_charge_rate_01, df_pid_tenant_name_mapping_01 = calculate_revenue(date_start_01, date_end_01, area_rentable_office_01, df_data_01, 'Office', 'Y')
df_sum_02, df_report_rental_charge_02, df_report_sc_charge_02, df_report_occupancy_02, df_report_sc_charge_rate_02, df_pid_tenant_name_mapping_02 = calculate_revenue(date_start_02, date_end_02, area_rentable_office_02, df_data_02, 'Office', 'Y')

# write to excel file
import xlwings as xw
wb = xw.Book(output_filename)
my_sheet = wb.sheets['TE Office (y)']
my_sheet.clear_contents()

df_to_show = df_sum_01.copy()

my_sheet.range('B3').value = df_to_show.transpose()
my_sheet.range('C4:M4').number_format = 'yyyy'
my_sheet.range('3:4').font.bold = True
my_sheet.range('B:B').font.bold = True
my_sheet.range('7:7').font.bold = True
my_sheet.range('C5:M8').number_format = '0,00#'
my_sheet.range('C9:M9').number_format = '0.00%'



# Display result dataframe (formatted)

In [16]:

format_mapping = {
    "date": "{:%Y}",
    "Rental": "Rp {:,.2f}",
    "SC": "Rp {:,.2f}",
    "Total": "Rp {:,.2f}",
    "Occ": "{:,.2f} m2",
    "OccPct": "{:,.2%}",
}

# print dataframe
df_sum_styled = df_sum_01.style.format(format_mapping)
#df_sum_styled['date'] = df_sum_styled['date'].dt.strftime("%Y")
df_sum_styled


,date,Rental,SC,Total,Occ,OccPct
0,2023,"Rp 202,464,385,194.32","Rp 42,958,440,470.72","Rp 245,422,825,665.04","48,164.74 m2",79.67%
1,2024,"Rp 225,318,057,665.27","Rp 48,224,646,579.37","Rp 273,542,704,244.63","53,745.10 m2",88.90%
2,2025,"Rp 225,358,633,395.27","Rp 49,295,911,066.82","Rp 274,654,544,462.09","53,741.18 m2",88.89%
3,2026,"Rp 224,363,567,815.27","Rp 49,423,744,829.31","Rp 273,787,312,644.57","53,480.77 m2",88.46%
4,2027,"Rp 221,122,710,950.31","Rp 48,775,127,578.40","Rp 269,897,838,528.71","52,760.08 m2",87.27%
5,2028,"Rp 227,555,428,860.56","Rp 51,037,542,069.16","Rp 278,592,970,929.72","54,005.50 m2",89.33%
6,2029,"Rp 229,774,350,418.78","Rp 50,515,531,959.20","Rp 280,289,882,377.98","53,087.64 m2",87.81%
7,2030,"Rp 235,732,165,186.99","Rp 50,666,432,047.20","Rp 286,398,597,234.19","53,254.95 m2",88.09%
8,2031,"Rp 239,353,736,386.99","Rp 52,464,423,181.34","Rp 291,818,159,568.33","54,028.79 m2",89.37%
9,2032,"Rp 239,353,736,386.99","Rp 52,816,124,759.39","Rp 292,169,861,146.38","54,028.79 m2",89.37%
